# SAE Comparison Experiments - Complete Workflow

This notebook provides a complete workflow for running SAE experiments from your GitHub repository.

**Steps:**
1. Setup environment
2. Clone repository
3. Install dependencies
4. Run experiments
5. Visualize results
6. Save to Google Drive

## 1. Setup Environment

In [ ]:
# Check GPU availability
import torch
import os

if torch.cuda.is_available():
    gpu_info = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"✓ GPU Available: {gpu_info}")
    print(f"  Memory: {gpu_memory:.1f} GB")
else:
    print("❌ No GPU detected!")
    print("Go to Runtime → Change runtime type → GPU")

In [ ]:
# Mount Google Drive (optional - for saving results)
from google.colab import drive
drive.mount('/content/drive')

# Create results directory in Drive
!mkdir -p /content/drive/MyDrive/sae_experiments

## 2. Clone Repository

In [ ]:
# Set your GitHub username
GITHUB_USERNAME = "YOUR_USERNAME"  # <-- Change this!
REPO_NAME = "mechanistic-interpretability-scale"

# Clone repository
!git clone https://github.com/{GITHUB_USERNAME}/{REPO_NAME}.git
%cd {REPO_NAME}

# Verify we're in the right directory
!pwd
!ls -la

## 3. Install Dependencies

In [ ]:
# Install from requirements.txt
!pip install -r requirements.txt -q

# Install package in development mode
!pip install -e . -q

print("✓ Dependencies installed!")

In [ ]:
# Verify imports
try:
    from src.models import KSparseSAE, GatedSAE, HybridSAE, VanillaSAE
    from src.training.trainer import SAETrainer
    from src.utils.model_loading import load_model
    print("✓ All imports successful!")
except Exception as e:
    print(f"❌ Import error: {e}")

## 4. Configure Experiment

In [ ]:
# Detect GPU and set appropriate configuration
if torch.cuda.is_available():
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    
    if gpu_memory > 30:  # A100
        config = {
            'model': 'gpt2-medium',
            'n_features': 16384,
            'batch_size': 32,
            'epochs': 10
        }
    elif gpu_memory > 15:  # V100/T4
        config = {
            'model': 'gpt2',
            'n_features': 8192,
            'batch_size': 16,
            'epochs': 5
        }
    else:  # Smaller GPUs
        config = {
            'model': 'gpt2',
            'n_features': 4096,
            'batch_size': 8,
            'epochs': 3
        }
    
    print(f"Configuration for {gpu_memory:.1f}GB GPU:")
    for key, value in config.items():
        print(f"  {key}: {value}")

## 5. Run Experiments

In [ ]:
# Quick test run (5-10 minutes)
!python experiments/01_sae_comparison/run_comparison.py \
    --model gpt2 \
    --layer 6 \
    --n_features 4096 \
    --batch_size 8 \
    --epochs 2 \
    --debug

In [ ]:
# Full experiment run (1-3 hours)
!python experiments/01_sae_comparison/run_comparison.py \
    --model {config['model']} \
    --layer 6 \
    --n_features {config['n_features']} \
    --batch_size {config['batch_size']} \
    --epochs {config['epochs']}

## 6. Visualize Results

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from IPython.display import Image, display

# Find results
results_dirs = sorted(Path('results').glob('sae_comparison_*'))
if results_dirs:
    latest_results = results_dirs[-1]
    print(f"Results found in: {latest_results}")
    
    # Load and display summary
    summary_path = latest_results / 'summary_results.csv'
    if summary_path.exists():
        df = pd.read_csv(summary_path)
        display(df)
    
    # Display all plots
    for img in sorted(latest_results.glob('*.png')):
        print(f"\n{img.name}:")
        display(Image(str(img)))
else:
    print("No results found yet!")

## 7. Save Results to Google Drive

In [ ]:
import shutil
from datetime import datetime

if results_dirs:
    # Create timestamped folder
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    drive_path = f'/content/drive/MyDrive/sae_experiments/run_{timestamp}'
    
    # Copy results
    shutil.copytree(latest_results, drive_path)
    print(f"✓ Results saved to: {drive_path}")
    
    # Also save the notebook
    !cp /content/{REPO_NAME}/notebooks/*.ipynb {drive_path}/

## 8. Advanced: Run Multiple Configurations

In [ ]:
# Run experiments with different settings
configurations = [
    {"model": "gpt2", "layer": 6, "n_features": 4096},
    {"model": "gpt2", "layer": 8, "n_features": 8192},
    {"model": "gpt2", "layer": 10, "n_features": 8192},
]

for i, conf in enumerate(configurations):
    print(f"\nRunning configuration {i+1}/{len(configurations)}...")
    !python experiments/01_sae_comparison/run_comparison.py \
        --model {conf['model']} \
        --layer {conf['layer']} \
        --n_features {conf['n_features']} \
        --batch_size 16 \
        --epochs 3

## 9. Clean Up (Optional)

In [ ]:
# Clear GPU memory
import gc
gc.collect()
torch.cuda.empty_cache()

# Check memory usage
!nvidia-smi